In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/heart-disease-uci/heart.csv") # importing data

In [ ]:
data.info() #şimdi de genel bir bilgi sahibi olalım

In [ ]:
data.columns

Şimdi bunlar neyi ifade ediyor, ona bakalım;
1. **age**:yaş
2. **sex**:cinsiyet-1 erkeği,sıfır ise kadını ifade ediyor
3. **cp**(chest pain):göğüs ağrısı (1:tipik angina 2:atipik angina 3:non-anginal ağrı 0:asemptomatik(ağrı yok))
4. **trestbps**(the person's resting blood pressure):kişinin dinlenim halinde ölçülen kan basıncı(mm-Hg)
5. **cho**l:kişinin mg/dl cinsinden kolesterol ölçümü
6. **fbs**(fasting blood sugar):kişinin açlık kan şekeri düzeyi(değerin 1 olması 120 mg/dl den yüksek olduğunu,0 ise 120 mg/dl den düşük olduğunu gösterir.)
7. **restecg**(resting electrocardiographic measurement):dinlenim durumundaki EKG değerlendirilmesi (0=normal,1=ST-T anormalileri bulunması(elevasyon/depresyon),2=sol ventrikülde hipertrofi bulgusunun kesin veya yüksek olasılıklı olması)
8. **thalach**: kişinin kalp atım hızının ulaştığı maksimum değer
9. exang:egzersiz indüklenmiş angina varlığı(1=var,0:yok)
10. **oldpeak**(ST depression induced by exercise relative to rest):dinlenme durumuna göre egzersizle artan ST depresyonu
11. **slop**e(the slope of the peak exercise ST segment ):maksimum egzersiz durumundaki ST segmenti eğimi
12. **ca**(number of major vessels): Fluoroscopy ile boyanmış gösterilen büyük damar sayısı(0-3)
13. **thal**(thalassemia):talasemi (1:normal,2:kalıcı defekt,3:geri dönüşümlü defekt)
14. **target**:kalp hastalığı(0:yok,1:var)


In [ ]:
data.head(10) #yukarıdan öğrendiklerimizle ilk 10 veriye bir göz atalım

In [ ]:
#sırada bu verilerin birbirleri ile ilişkilerinin incelenmesi var
data.corr()

In [ ]:
#bunu görselleştirelim - correlation map
f,ax = plt.subplots(figsize = (18,18))
sns.heatmap(data.corr(),annot=True,linewidths=.5,fmt=".2f",ax=ax,cbar=True,linecolor="w")#cbar kenardaki göstergenin varlığını belirler
plt.show()

Burada doğru orantılı olarak en yüksek ilişki içinde olan iki değer cp(göğüs ağrısı) ve target(hastalığın varlığı) iken birbirleriyle ters orantılı olarak ilişkili olan değerler içinde ise en fazla göze çarpanı slop(ST segmenti eğimi) ve oldpeak tir. Ancak bu grup hem hasta olan hem de hasta olmayan bireyleri içerdiğinden yukarıdaki grafik ile bir sonuca ulaşmamız pek mümkün değildir.

<a id="1"></a> <br>
# 1.INTRODUCTION TO PYTHON

In [ ]:
# line plot
# önce üzerinde çalışılan grubun yaş dağılımına bakalım
data.age.plot(kind="line",color="r",label="age",linewidth=2,alpha=1,grid=True,linestyle=":",figsize=(15,8))
plt.legend(loc=0)# loc=0 yerine 'best' de diyebilirdik. bu label ı en uygun yere koyar.
plt.xlabel("index")
plt.ylabel("age")
plt.title("yaş dağılımı")
plt.show()

Buradan gördüğümüz kadarıyla incelediğimiz grup büyük çoğunlukla 40 yaş üstü bireyleri barındırıyor.
Şimdi de histogram ile yaşların frekanslarına bakarak buna daha sayısal bir şekilde karar verelim.

In [ ]:
#Histogram
data.age.plot(kind="hist",bins=25,figsize=(12,8))
plt.show()

* Genel olarak haklı çıktığımızı söyleyebiliriz.

In [ ]:
# Açlık kan şekeri yüksek olan kişi sayısını bulalım
len(data[data.fbs == 1])

* Gruptaki 45 kişinin açlık kan şekeri yüksekmiş.
* Şimdi de kan basıncı 120 mm-Hg den yüksek olanları bulalım

In [ ]:
len(data[data.trestbps>120])

* Hem fbs>120 hem de trestbps>120 olan kişilerin hastalık bulundurma oranına bakalım

In [ ]:
dataft = data[(data.trestbps > 120) & (data.fbs == 1)]
dataft.info()
dataft[dataft.target==1].info()

* Bu özellikleri taşıyan 39 kişiden 18i hastaymış.

In [ ]:
# scatter plot - trestbps ve chol arasındaki ilişkiye bakılması
# x=trestbps y=chol
data.plot(kind="scatter",x="trestbps",y="chol",color="red",alpha=0.5,figsize=(10,5))
plt.xlabel("trestbps")
plt.ylabel("chol")
plt.title("trestbps-chol ilişkisi")
plt.show()

* Hastalığı bulunduranları bir dataya eşitleyip biraz da onun üstünde çalışalım

In [ ]:
data2 = data[data.target==1]
data2

In [ ]:
#genel bir bilgi sahibi olalım
data2.describe()

* Hasta olan grubun yaş ortalaması 52,5 tur. Bu grupta hastalığın görüldüğü en geç yaştaki kişi 29 yaşında iken hastalığın görüldüğü en yaşlı kişi 76 yaşındadır.
* Hasta olan grubun dinlenim kan basıncı ortalama 129.3 tür.
* Hasta grupta kalp atışlarının ulaştığı max değer ortalama 158 dir.

In [ ]:
# Hasta grupta egzersiz anjinasının görülme sıklığına bakalım şimdi de
data2.exang.plot(kind="hist",bins=10,color="c",figsize=(8,5))
plt.show()

* Şimdi de hastalıkta cinsiyetin bir rolü var mı bakalım
* 1=erkek 0=kadın

In [ ]:
data2.sex.plot(kind="hist",bins=10,color="m")
plt.show()

* Bu grupta görüldüğü kadarıyla hastalığın erkeklerde görülme sıklığı kadınlarda görülme sıklığından daha fazladır.

### Dictionary

In [ ]:
#sözlük oluşturma
sozluk = {"Türkiye":"TL","İngiltere":"sterlin"}
print(sozluk.keys())
print(sozluk.values())
print(sozluk.items())

In [ ]:
sozluk["İngiltere"] = "ingiliz sterlini" # var olan entryi güncelledik
print(sozluk)
sozluk["USA"] = "USD" # yeni bir entry ekledik
print(sozluk)
del sozluk["İngiltere"] # entryi sildik
print(sozluk)
print("USA" in sozluk) # yazımından da anlaşılacağı gibi sözlükte bulunup bulunmadığını söyler
sozluk.clear() # sözlükteki bütün entryleri siler
print(sozluk)
# del sozluk komutu ise sözlüğü tamamen silecektir

### PANDAS

In [ ]:
# data = pd.read_csv("../input/heart-disease-uci/heart.csv") kodu ile datayı yukarıda import etmiştik

In [ ]:
series = data['age']        
print(type(series))        # series-dataframe ilişkisi numpy daki vector-array ilişkisine benzer
dataframe = data[['age']] 
print(type(dataframe))


### WHİLE AND FOR LOOPS

In [ ]:
# thal = 2 (kalıcı defekt) olan kişi sayısını bulalım
liste_thal = []
for i in data.thal:
    if(i==2):
        liste_thal.append(i)
print(len(liste_thal))

In [ ]:
# while döngüsü ile ufak bir sayı tahmin oyunu yapalım
deneme_hakki = 0 #sonraki blokların kesintiye uğramadan çalışması için deneme hakkını 0 yaptım. Siz çalıştırmak için 3 yapabilirsiniz.
xyz = np.random.randint(0,5)
while deneme_hakki>0:
    sayi = int(input("Bir sayı giriniz: ")) # input fonksiyonu ile aldığımız değer string olur,buna dikkat etmezsek kodumuz çalışmayacaktır.
    if (xyz==sayi):
        print("Tebrikler, doğru bildiniz. Sayımız: {} ".format(sayi))
        break
    else:
        deneme_hakki -= 1
        print("yanlış tahmin")
    if deneme_hakki == 0:
        print("denem hakkınız tükendi")

In [ ]:
# enumerate fonksiyonu
lis = ["a","b","c","d","e"]
for index,value in enumerate(lis):
    print(index," : ",value)
print("\n") # \n bir alt satıra geçmeyi sağlar.print fonksiyonunda bu zaten vardır,her seferinde bir alt satıra geçer.o yüzden burada iki alt satıra geçti.
# sözluklerde items metodu
sozluk = {"a":"x","b":"y"}
for key,value in sozluk.items():
    print(key," : ",value)
print("\n")
# iterrows
for index,value in data[["cp"]][0:3].iterrows():
    print(index," : ",value)

In [ ]:
for i,j in data.iterrows():   # burada i satır indeksini, j ise satırı ifade eder. İlk satırı alalım
    print("i: ",i," j: ",j)
    break

In [ ]:
# thalach değeri ortalama değerden fazla olanlarla bir dataframe oluşturalım
data_thalach = pd.DataFrame()
for i,j in data.iterrows():
    if(j.thalach>data.thalach.mean()):
        data_thalach = data_thalach.append(j)
data_thalach.info()

In [ ]:
#son olarak da yeni dataframimizi görselleştirelim
data_thalach.thalach.plot(kind="line",color="m",label="thalach",linewidth=1,alpha=0.9,grid=True,linestyle="--",figsize=(12,8))
plt.legend(loc=0)
plt.xlabel("index")
plt.ylabel("thalach")
plt.title("thalach değeri ortalamadan büyük olanlar")
plt.show()